In [2]:
import pandas as pd
import csv
import math
import networkx as nx 
import heapq
import random 
# numpy pour dijkstra 
import numpy as np
from numpy import Inf

#Widget
import ipywidgets as widgets
from IPython.display import display

#Map interactive
import folium as f
import folium
from folium.plugins import MarkerCluster

#Map pour connection
import matplotlib.pyplot as plt
%matplotlib inline

#Calcul great circle
import pyproj

import tkinter
#import os
#import flightradar24
from branca.element import Figure
import igraph as ig

from IPython.core.display import HTML
from ipywidgets import Output
import ipyvuetify as v

In [3]:
#pip install markupsafe==2.1.1

In [4]:
#pip install pandas
#pip install networkx
#pip install ipywidgets
#pip install folium
#pip install matplotlib
#pip install pyproj
#pip install nbinteract
#pip install scipy
# pip3 install nbdev
#python -m pip install markupsafe==2.0.1

In [5]:
#python -m pip install markupsafe==2.0.1

In [6]:
#pip install voila
# pip install networkx
# pip install pyproj

# pyhton -m pip install package 

Init data 

In [7]:
#chemin="../data/"
chemin="../../data/"

#df_aeroports = pd.read_csv(chemin + "aeroportsANLO.csv")
#df_zones = pd.read_csv(chemin + "zonesANLO.csv")
df_vols = pd.read_csv(chemin + "routesANLO.csv")
#df_compagnies = pd.read_csv(chemin + "compagniesANLO.csv")

In [8]:
dict_compagnies = {}

with open(chemin+'compagniesANLO.csv', mode='r') as inp:
    reader = csv.reader(inp)
    next(reader, None)
    headers = ['Name','Country']
    dict_compagnies = {rows[2]:dict(zip(headers,[rows[0],rows[3]])) for rows in reader}


In [9]:
dict_aeroports = {}

with open(chemin+'aeroportsANLO.csv', mode='r') as inp:
    reader = csv.reader(inp)
    headers = next(reader, None)
    dict_aeroports = {rows[1]:dict(zip(headers,rows[0:8])) for rows in reader}

dict_aeroports['JFK']
dict_aeroports['EWR']
dict_aeroports['LGA']
#DistGrandCercleICAO('JFK', 'EWR') < 50 

{'Name': 'New York LaGuardia Airport',
 'Iata': 'LGA',
 'Icao': 'KLGA',
 'Lat': '40.777248',
 'Lon': '-73.872597',
 'Country': 'United States',
 'Alt': '21',
 'City': 'New York'}

In [10]:
dict_pays = {}

with open(chemin+'countries.csv', mode='r') as inp:
    reader = csv.reader(inp)
    headers = next(reader, None)
    dict_pays = {rows[1]:rows[0] for rows in reader}

#dict_pays

In [11]:
df_vols = df_vols[["Depart","Arrivee","Code_ICAO"]].dropna()

In [13]:
def coordDepuisAeroport(iata : str):
    '''
    Fonction : Recherche les coordonnées GPS d'un aéroport
    Retour : lat : float ,lon : float --> Latitude & longitude de l'aéroport
    '''
    lat = float(dict_aeroports[iata]['Lat'])
    lon = float(dict_aeroports[iata]['Lon'])
    return lat,lon

def DistGrandCercle(lat1 : float, lon1 : float, lat2 : float, lon2 : float):
    #Conversion en radian
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    #Delta des coordonnées
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Rayon terrestre (en km)
    r = 6367.0

    #Formule d'Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return 2 * r * math.asin(math.sqrt(a))


def DistGrandCercleICAO(icao1 : str, icao2 : str):
    #Récupération des coordonnées des aéroports
    latDep, lonDep = coordDepuisAeroport(icao1)
    latArr, lonArr = coordDepuisAeroport(icao2)

    #On calcule la distance les séparant,                                                     
    return DistGrandCercle(latDep,lonDep,latArr,lonArr)

def Sqr(a):
    return a*a

def Dist(x1,y1,x2,y2):
    return math.sqrt((y2-y1)**2+(x2-x1)**2)

In [14]:
import json 
import pickle

In [16]:
a_file = open("dict_route.pkl", "rb")
dict_route = pickle.load(a_file)

Fonctions 

In [17]:
def filtreDictAeroports(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des aéroports
    '''
    Listtmp = []
    for (key, value) in dict_aeroports.items():
        if value[colonne] == valeur:
            Listtmp.append(key)
        
        if valeur == '':
            Listtmp.append(value[colonne])
    return Listtmp

def filtreDictCompagnies(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des compagnies
    '''
    Listtmp = []
    for (key, value) in dict_compagnies .items():
        if value[colonne] == valeur:
            Listtmp.append(key)
            
        if valeur == '':
            Listtmp.append(value[colonne])
    return Listtmp

In [18]:
def correspondanceAirport(code1 : str, code2 : str):
    '''
    fonction qui filtre le df_vols pour obtenir les correspondances
    retour : list des airports codes 
    '''
    r1 = df_vols[df_vols['Depart'] == code1]['Arrivee'].to_list()
    r2 = df_vols[df_vols['Arrivee'] == code2]['Depart'].to_list()

    return list(set(r1).intersection(r2))

In [19]:
def memeContinent(str_air1, str_air2):
    '''
    fonction qui retourne vrai si les deux sont sur le meme continent 
    et faux sinon
    '''
    c1 = dict_aeroports[str_air1]['Country']
    c2 = dict_aeroports[str_air2]['Country']
    
    return c1 == c2

In [27]:
# copie de loris 
def vectGrandCercle(latDep : float, lonDep : float, latArr : float, lonArr : float):
    '''
    Fonction : Calcul un semble de coordonnées permettant de tracer un grand cercle terrestre
    Return : 2 listes de coordonnées
    '''
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(lonDep, latDep, lonArr, latArr)

    # Coordonnée avec des segments <= 100 km
    lonlats = g.npts(lonDep, latDep, lonArr, latArr,
                    1 + int(dist / 100000))

    #Mise en forme
    v1 = []
    v1.append([latDep,lonDep])
    v2 = []
    _lon = lonDep
    _lat = latDep
    horscadre = False
    for lon, lat in lonlats:
        if(not(horscadre) and Dist(_lat,_lon,lat,lon) < 4):        
            v1.append([lat,lon])
        else:
            horscadre = True
            v2.append([lat,lon])
        _lon = lon
        _lat = lat
    
    if(horscadre):
        v2.append([latArr,lonArr])
    else:
        v1.append([latArr,lonArr])

    return v1,v2

In [28]:
def creation_lines(path : list, Line_group, c):
    '''
    fonction qui cree les lignes pour le folium map
    path c'est le chemin, line_group c'est le group commun, c c'est le couleur de ligne 
    '''
    for element in range(len(path)-1) :  
        start = path[element]
        end = path[element+1]
        latDep, lonDep = coordDepuisAeroport(start)
        latArr, lonArr = coordDepuisAeroport(end)

        #Calcul des chemins à afficher
        v2 = []
        v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)
        
        col_ligne = c
        col_marker = c 
        
        #-------------------
        folium.PolyLine(locations=v1,weight=3, color=col_ligne).add_to(Line_group)
        if(len(v2) != 0):
            folium.PolyLine(locations=v2,weight=3, color=col_ligne).add_to(Line_group)

        folium.CircleMarker(location = [latArr, lonArr],
                        radius = 1, 
                        color = col_marker, 
                        tooltip=dict_aeroports[end]['City'],
                        popup="<b>Nom : </b>" + dict_aeroports[end]['Name'] + """<br />
                                <b>Code :</b> """ + dict_aeroports[end]['Iata'] + """<br />
                                <b>Coordonnées :</b> (""" + str(dict_aeroports[end]['Lat']) + ";" + str(dict_aeroports[end]['Lon']) + """)<br />
                                <b>City : </b>""" + str(dict_aeroports[end]['City']) + """<br />
                                <b>Pays : </b>""" + dict_aeroports[end]['Country']).add_to(Line_group)

fonction - map

In [29]:
outputMAPPlusieurs = Output()
outputMAPCourt = Output()


def map_correspondance(collection_path, b : bool):
    '''
    fonction qui gere le ajoute de chaque ligne pour tous les chemins qui sont dans le collection path 
    collection sous la forme de {0 : {'distance' : .. , 'path' : []}, 1 : {..} , etc}
    '''
    
    # 1 group de line 
    Line_group = folium.FeatureGroup(name = "Tous")

    # generer les differents couleurs pour les differents chemins 
    longeur = 0 
    if isinstance(collection_path, dict) : longeur = len(collection_path.keys())
    if isinstance(collection_path, list) : longeur = len(collection_path)
    rand_colors = []
    for _ in range(longeur):
        rand_colors.extend( ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])] ) 
    
    # ajoute le marker du debut 
    air_start = str
    if isinstance(collection_path, dict) : air_start = collection_path[0]['path'][0]
    if isinstance(collection_path, list) : air_start = collection_path[0][0]
    latStart, lonStart = coordDepuisAeroport(air_start)
    
    m = folium.Map(location=[latStart, lonStart], zoom_start=2)
    
    folium.CircleMarker(location = [latStart, lonStart],
                        radius = 1, 
                        color = '#000000', 
                        tooltip=dict_aeroports[air_start]['City'],
                        popup="<b>Nom : </b>" + dict_aeroports[air_start]['Name'] + """<br />
                                <b>Code :</b> """ + dict_aeroports[air_start]['Iata'] + """<br />
                                <b>Coordonnées :</b> (""" + str(dict_aeroports[air_start]['Lat']) + ";" + str(dict_aeroports[air_start]['Lon']) + """)<br />
                                <b>City : </b>""" + str(dict_aeroports[air_start]['City']) + """<br />
                                <b>Pays : </b>""" + dict_aeroports[air_start]['Country']).add_to(Line_group)
    
    
    if isinstance(collection_path, dict) : 
        # iteration sur le dict pour ajouter chaque chemin 
        for key, contenu in collection_path.items() : 
            # une couleur par chemin 
            couleur = rand_colors.pop()
            # creation de poly lignes 
            creation_lines(contenu['path'], Line_group, couleur)
        
        
        
    if isinstance(collection_path, list):
        for contenu in collection_path : 
            # une couleur par chemin 
            couleur = rand_colors.pop()
            # creation de poly lignes 
            creation_lines(contenu, Line_group, couleur)
        
    #Ajout les lines à la carte
    Line_group.add_to(m)
    
    fig = Figure(width='100%', height=None, ratio='60%')
    fig.add_child(m)

    if b : 
        with outputMAPPlusieurs: 
            display(fig)
    else : 
        with outputMAPCourt:
            display(fig)


# Construction graphe totale

In [31]:
array_arrivee = []
array_depart = [] 
for key in dict_route:

    if len(key) == 2 :
        depart, arrive = key 
        
        array_arrivee.append(arrive)
        array_depart.append(depart)

list_arrivee = np.unique(array_arrivee)
list_depart = np.unique(array_depart)

list_arrivee = list(map(lambda x : dict_aeroports[x]['Name'], list_arrivee))
list_depart = list(map(lambda x : dict_aeroports[x]['Name'], list_depart))

list_arrivee = sorted(list_arrivee)
list_depart = sorted(list_depart)


In [32]:
graph_totale = nx.Graph()
list_airports_codes = []; 

compteur = 0 
list_debug = [] 
for key in dict_route:

    if len(key) == 2 :
        start, end = key 
        compteur = compteur + 1
        
        if list_airports_codes.count(start) == 0:
            list_airports_codes.append(start)

        if list_airports_codes.count(end) == 0:
            list_airports_codes.append(end)
            
        index_start = list_airports_codes.index(start); 
        index_end = list_airports_codes.index(end); 
        distance = dict_route[frozenset((start,end))]['Dist']
        
        graph_totale.add_node(index_start)
        graph_totale.add_node(index_end)
        
        graph_totale.add_edge(index_start, index_end, weight = distance)
    else: list_debug.append(key)

nb_keys = len(dict_route.keys())
#print('Nombre de keys dans le dict_route : ', print(nb_keys))
debug = nb_keys - compteur
#print('Nombre de keys qui ne sont pas ajoute : ', debug , ', voici le tableau avec le keys qui ne sont pas ajoutee')
#print(list_debug)

In [ ]:
def dijkstra(graphe : nx.Graph, start : int, end : int):
    '''
        fonction qui utilise l'algorithme de Dijkstra pour calcule le plus court
        chemin et calcule le distance
        retour \: dict{\'distance\' : ... , \'path\' : ...}
    '''
    try :
        tmp1 = nx.dijkstra_path(graphe, start,end)
        dist = nx.dijkstra_path_length(graphe, start, end)
        return {'distance' : dist, 'path' : tmp1}
    except nx.NetworkXNoPath:
        print('no path (dijkstra)')
        return -1 

In [33]:
def supprimerJusqauNombre(g_modif : nx.Graph, nombre : int, index1 , index2):
    
    index_compteur = 0 
    collection_solutions = dict() 
    
    solution = dijkstra(g_modif, index1, index2)
    if(solution == -1):
        print('No path (supprimerJusquaNombre)')
        return -1 
    else : 
        collection_solutions[index_compteur] = solution; 
        
        memoire = []
        
        while(len(solution['path']) < nombre ):
            
            ###################################
            for i in range(len(solution['path'])-1) :
                w = g_modif.get_edge_data(solution['path'][i], solution['path'][i+1])['weight']
                memoire.append([solution['path'][i], solution['path'][i+1], w])
                
                # delete les paths le plus vite --> on trouve les autres paths plus vite 
                g_modif.remove_edge(solution['path'][i], solution['path'][i+1])
            
            ###################################
            # trouve la nouvelle solution 
            solution = dijkstra(g_modif, index1, index2)
            if(solution == -1):
                print('No path (supprimerJusquaNombre)')
                return -1 
            else : 
                collection_solutions[index_compteur] = solution; 
                # ajouter la solution au dict de resultat 
                collection_solutions[index_compteur] = solution
                # augementer le compteur ( qui fonctionne comme cle dans le dict)
                index_compteur = index_compteur + 1 

        # remettre le graph au graph originale 
        g_modif.add_weighted_edges_from(memoire)
    
    return collection_solutions    

In [34]:
def plus_courte(index1, index2, airport1, airport2): 
    if graph_totale.has_node(index1):
        if graph_totale.has_node(index2):
            
            ###################### plus courtes ###########################
            
            resultat = dijkstra(graph_totale, index1, index2)
            if(resultat == -1 ):
                print('no path (plus_courte)')
                return -1
            else : 
                s = list(map(lambda x : list_airports_codes[x], resultat['path']))
                solution_plus_courte = dict(); 
                solution_plus_courte[0] = {'distance' : resultat['distance'], 'path' : s }
                print(solution_plus_courte)
                map_correspondance(solution_plus_courte, False)
                return solution_plus_courte
        else : print(airport1, ' n\'est pas presente dans le graphe')
    else : print(airport2, ' n\'est pas presente dans le graphe')

In [35]:
def differents_longeurs(index1, index2, airport1, airport2, list_nombre):
    if graph_totale.has_node(index1):
        if graph_totale.has_node(index2):
            
            
            ###################### differentes longeurs ###################
            # TODO minder verwijderen 
            collection_solution = supprimerJusqauNombre(graph_totale, list_nombre, index1, index2)
            if( collection_solution == -1 ):
                print('No path (differents longeurs)')
                return -1
            else : 
                new_collection_solution = dict() 
                c = 0 
                for key, contenu in collection_solution.items():
                    
                    if len(contenu['path']) <= list_nombre: #and len(contenu['path']) >= int_s : 
                        tmp = []
                        for x in contenu['path'] :
                            tmp.append(list_airports_codes[x])
                        
                        new_collection_solution[c] = {'distance' : contenu['distance'], 'path' :tmp}  
                        c = c + 1 
            
                print('Collection : ', new_collection_solution)
                
                map_correspondance(new_collection_solution, True)    
                
        else : print(airport1, ' n\'est pas presente dans le graphe')
    else : print(airport2, ' n\'est pas presente dans le graphe')

In [36]:
def response_widget(nom1, nom2, nombre):
    
    airport1 = filtreDictAeroports('Name', nom1)[0]
    airport2 = filtreDictAeroports('Name', nom2)[0] 

    if not(list_airports_codes.count(airport1) == 0) : 
        index1 = list_airports_codes.index(airport1)
    else : print(nom1, 'n\'est pas dans le graphe')
    if not(list_airports_codes.count(airport2) == 0) : index2 = list_airports_codes.index(airport2)   
    else : print(nom2, 'n\'est pas dans le graphe') 

    resultat = plus_courte(index1, index2, airport1, airport2)
    
    if resultat != -1 and nombre >= len(resultat[0]['path']) : 
        differents_longeurs(index1, index2, airport1, airport2, nombre)
    else : print('Le nombre que vous avez choisi c\'est trop petit')

## conversion de resultat en dataframe --> print tableau dans le vuefity 

In [37]:
test = {0: {'distance': 623.5331324194632, 'path': ['AMS', 'AAL']}, 1 : {'distance': 700, 'path': ['LYS', 'AAL']}}

dataFrameOutPut = pd.DataFrame(columns = ['Depart', 'Arrive', 'Distance'])

for item in test.items() :
    item = item[1]
    dataFrameOutPut.loc[len(dataFrameOutPut.index)] = [dict_aeroports[item['path'][0]]['Name'], dict_aeroports[item['path'][1]]['Name'], item['distance']]
    
dataFrameOutPut

    

,Depart,Arrive,Distance
0,Amsterdam Schiphol Airport,Aalborg Airport,623.533132
1,Lyon Saint Exupery Airport,Aalborg Airport,700.000000


# Vuetify 

In [38]:
airportD = v.Select(
    
    label = 'Airport de depart', 
    items = list_depart, 
    thumb_label='always', 
    v_model = "Amsterdam Schiphol Airport"
    )

airportA = v.Select(
    
    label = 'Airport d\'arrivee', 
    items = list_arrivee, 
    thumb_label='always', 
    v_model= "Los Angeles International Airport"
    )

nbAirports = v.Select(
    
    label = 'Combien de airports en totale?', 
    items = [2,3,4,5,6,7,8,9], 
    thumb_label='always', 
    v_model=5
    )

choix = v.Container(children = [airportD, airportA, nbAirports])

In [39]:
def on_click(widget, event, data):
    # ces deux fonctions enleve l'ouput seulement si il y a quelque chose nouvelle a dessiner 
    outputMAPCourt.clear_output(wait = True )
    outputMAPPlusieurs.clear_output(wait = True)
    # appele du fonction 
    response_widget(airportD.v_model, airportA.v_model, nbAirports.v_model)
    

airportD.on_event('change', on_click)
airportA.on_event('change', on_click)
nbAirports.on_event('change', on_click)

# pour initialiser l'interface initiale 
response_widget(airportD.v_model, airportA.v_model, nbAirports.v_model)

{0: {'distance': 8950.27748845727, 'path': ['AMS', 'LAX']}}
Collection :  {0: {'distance': 8952.187224431127, 'path': ['AMS', 'SLC', 'LAX']}, 1: {'distance': 8971.405037398723, 'path': ['AMS', 'LAS', 'LAX']}, 2: {'distance': 9025.430569854188, 'path': ['AMS', 'EMA', 'LAX']}, 3: {'distance': 9071.813796647712, 'path': ['AMS', 'STN', 'LAX']}, 4: {'distance': 9103.932065688443, 'path': ['AMS', 'YYC', 'LAX']}, 5: {'distance': 9124.285677604237, 'path': ['AMS', 'LHR', 'LAX']}, 6: {'distance': 9135.616387016526, 'path': ['AMS', 'YEG', 'LAX']}, 7: {'distance': 9146.366508644252, 'path': ['AMS', 'MSP', 'LAX']}, 8: {'distance': 9149.738531646608, 'path': ['AMS', 'FAB', 'LAX']}}


## widgets loris 

### map connection 

In [40]:
# creation du list a partir laquelle on peut choisir 
list_Compagnies = filtreDictCompagnies('Name', '')

In [120]:
compagnies = v.Select(
    label = 'Compagnie *', 
    items = list_Compagnies, 
    thumb_label='always', 
    v_model = "Air France",
    class_="px-6"
)

depart = v.Select(
    label = 'Airport de depart', 
    items = list_depart, 
    thumb_label='always', 
    v_model = "Amsterdam Schiphol Airport",
    class_="px-6"
)

tousLesCompagnies = v.Checkbox(
    v_model = False, 
    label = "tous les compagnies",
    thumb_label='always', 
    class_="px-6",
    width = '50%', 
    align_center = True 
)

outputMapConnection = Output() 

In [42]:
def map_connection(depart,compagnie,m):
    Line_monde = f.FeatureGroup(name = "International")
    Line_continent = f.FeatureGroup(name = "Continental")
    Line_pays = f.FeatureGroup(name = "National")   

    
    df_tmp = df_vols[df_vols['Depart'] == depart]

    for _, ligne in df_tmp.iterrows():
        #Certains aéroports n'ont pas de correspondance dans le dict. Pour éviter l'erreur -> try puis pass
        try:
            latDep, lonDep = coordDepuisAeroport(ligne.Depart)
            latArr, lonArr = coordDepuisAeroport(ligne.Arrivee)

            #Calcul des chemins à afficher
            v2 = []
            v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)

            #Test vol national ou international
            if(dict_aeroports[ligne.Depart]['Country'] == dict_aeroports[ligne.Arrivee]['Country']):
                Line_group = Line_pays
                col_ligne = "#80F0A3"
                col_marker = "#008329"
            elif(dict_pays[dict_aeroports[ligne.Depart]['Country']] == dict_pays[dict_aeroports[ligne.Arrivee]['Country']] ):
                Line_group = Line_continent
                col_ligne = "#F6FF28"
                col_marker = "#FE7700"
            else:
                Line_group = Line_monde
                col_ligne = "#563DFF"
                col_marker = "#0004A8"


            #-------------------
            f.PolyLine(locations=v1,weight=1, color=col_ligne).add_to(Line_group)
            if(len(v2) != 0):
                f.PolyLine(locations=v2,weight=1, color=col_ligne).add_to(Line_group)
            
            f.CircleMarker(location = [latArr, lonArr],
                            radius = 1, 
                            color = col_marker, 
                            tooltip=dict_aeroports[ligne.Arrivee]['City'],
                            popup="<b>Nom : </b>" + dict_aeroports[ligne.Arrivee]['Name'] + """<br />
                                    <b>Code :</b> """ + dict_aeroports[ligne.Arrivee]['Iata'] + """<br />
                                    <b>Coordonnées :</b> (""" + str(dict_aeroports[ligne.Arrivee]['Lat']) + ";" + str(dict_aeroports[ligne.Arrivee]['Lon']) + """)<br />
                                    <b>City : </b>""" + str(dict_aeroports[ligne.Arrivee]['City']) + """<br />
                                    <b>Pays : </b>""" + dict_aeroports[ligne.Arrivee]['Country']).add_to(Line_group)
        except:
            pass

    #Ajout des layers à la carte
    Line_monde.add_to(m)
    Line_continent.add_to(m)
    Line_pays.add_to(m)

def on_change_ma_connection(widget, event, data):
    
    #Paramètre de la carte
    m = f.Map(
        location = [0, 0],
        zoom_start = 2.3,
        min_zoom  = 2,
        min_lot=-180,
        max_lot=180,
        min_lat=-90,
        max_lat=90,
        max_bounds=True,
    )
    f.TileLayer('cartodbdark_matter').add_to(m)
    
    code_compagnie = filtreDictCompagnies('Name', compagnies.v_model)
    code_depart = filtreDictAeroports('Name', depart.v_model)[0]
    
    if tousLesCompagnies.v_model :
        map_connection(code_depart, "" ,m)
    else : 
        map_connection(code_depart, code_compagnie ,m)
        
    f.LayerControl().add_to(m)
    
    fig = Figure()
    fig.add_child(m)
    outputMapConnection.clear_output(wait = True)
    with outputMapConnection:
        display(fig)

compagnies.on_event('change', on_change_ma_connection)
depart.on_event('change', on_change_ma_connection)
tousLesCompagnies.on_event('change', on_change_ma_connection)

# initialisation 
on_change_ma_connection('', '', '')

### map connection pays

In [43]:
list_pays = list(dict_pays.keys())

w_pays_li = v.Select(
    label = 'Pays', 
    items = list_pays, 
    thumb_label='always', 
    v_model = "France",
    class_="px-6"
)

outputMapConnectionPays = Output()

In [44]:

def map_connectionPays(pays,m):
    '''
    Fonction : Dessine les liaisons entre différentes aéroports d'un même pays
    Return : None
    '''
    
    #Code ICAO des aéroports dans le pays en question
    listAeroportPays = filtreDictAeroports('Country',pays)

    #Filtrage des vols en fonction des aéroports possibles
    df_volsFiltre = df_vols[df_vols["Depart"].isin(listAeroportPays) & df_vols["Arrivee"].isin(listAeroportPays)]

    #Nombre de vols par aéroport (Depart + Arrivee)
    Serie_nbrVolsParAeroport =  pd.concat([df_volsFiltre['Depart'],df_volsFiltre['Arrivee']]).value_counts()
    #On groupe par liaison puis on on ajoute la colonne 'count' pour savoir combien de vol par liaison
    df_GroupLiaison = df_volsFiltre.groupby(['Depart','Arrivee']).size().reset_index(name="Count")
    
    Serie_nbrDestinationParAeroport =  df_GroupLiaison['Depart'].value_counts()


    maxLigne = df_GroupLiaison['Count'].max()
    minLigne = df_GroupLiaison['Count'].min()

    #print(maxLigne)
    #print(minLigne)
    set1=set()

    #Création des Layers
    tabLine = []
    _max = maxLigne/5
    #-0.1 pour ne pas créer un nouveau bloc en cas de division entière
    for i in range(0,int(_max-0.1) + 1):
        tabLine.append(f.FeatureGroup(name = "Entre " + str(int(i*5) + 1) + " et " +
                                                str((int((i+1)*5))) + " liaisons"))

    for _, ligne in df_GroupLiaison.iterrows():
        #Certains aéroports n'ont pas de correspondance dans le dict. Pour éviter l'erreur -> try puis pass
        try:
            latDep, lonDep = coordDepuisAeroport(ligne.Depart)
            latArr, lonArr = coordDepuisAeroport(ligne.Arrivee)

            #Calcul des chemins à afficher
            v2 = []
            v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)
            
            #print(v1)
            #-------------------
            set1.add(ligne.Count/5)

            f.PolyLine(locations=v1,weight=0.2 +ligne.Count/5, color='blue', opacity=0.5,
                        popup="<b>" + dict_aeroports[ligne.Depart]['City'] + " --> " +
                                dict_aeroports[ligne.Arrivee]['City'] +  "</b><br/>" +
                                "<b>Liaisons : </b>" + str(ligne.Count)
                        ).add_to(tabLine[int(ligne.Count/5 - 0.1)])
            
            if(len(v2) != 0):
                f.PolyLine(locations=v2,weight=0.2 +ligne.Count/5, color='blue', opacity=0.5,
                        popup="<b>" + dict_aeroports[ligne.Depart]['City'] + " --> " +
                                dict_aeroports[ligne.Arrivee]['City'] +  "</b><br/>" +
                                "<b>Liaisons : </b>" + str(ligne.Count)
                        ).add_to(tabLine[int(ligne.Count/5 - 0.1)])
            #print(ligne.Depart, " : ",df_GroupDepart[ligne.Depart])
        except:
            print("Erreur entre ", ligne.Depart, " et ", ligne.Arrivee, " -- ", ligne.Count)
            pass
    
    #Ajout des Layers:
    for i in range(0,int(_max-0.1) + 1):
        tabLine[i].add_to(m)

    for ligne in Serie_nbrVolsParAeroport.items():

        if ligne[0] in Serie_nbrDestinationParAeroport.index:
            nbrDest = Serie_nbrDestinationParAeroport[ligne[0]]
        else:
            nbrDest = 0  
        
        Text = f.IFrame("<b>" + dict_aeroports[ligne[0]]['Name'] + """</b><br />
                        <b>City : </b>""" + str(dict_aeroports[ligne[0]]['City']) + """<br />
                        <b>Pays : </b>""" + dict_aeroports[ligne[0]]['Country'] + """<br />
                        <b>Code :</b> """ + dict_aeroports[ligne[0]]['Iata'] + """<br/><br/>
                        <b>Destinations : </b> """ + str(nbrDest))

        #Circle vs CircleMarker
        f.CircleMarker(location= [dict_aeroports[ligne[0]]['Lat'],dict_aeroports[ligne[0]]['Lon']], 
                        radius = 1 + ligne[1]/35, 
                        color='red', 
                        fill_color='red',
                        fill_opacity=1,
                        tooltip=dict_aeroports[ligne[0]]['Name'],                        
                        popup = f.Popup(Text, min_width = 250, max_width = 400)
                        ).add_to(m)       

def on_change_ma_connection_pays(widget, event, data):
    
    #Paramètre de la carte
    m1 = f.Map(
        location = [46, 2],
        zoom_start = 6,
        min_zoom  = 2,
        min_lot=-200,
        max_lot=200,
        min_lat=-90,
        max_lat=90,
        max_bounds=True,
    )
    #f.TileLayer('cartodbdark_matter').add_to(m)
    
    map_connectionPays(w_pays_li.v_model, m1)
    #f.PolyLine(locations=line).add_to(m)
    f.LayerControl().add_to(m1)
    
    li = Figure()
    li.add_child(m1)
    outputMapConnectionPays.clear_output(wait = True)
    with outputMapConnectionPays:
        display(li)

w_pays_li.on_event('change', on_change_ma_connection_pays)
t = on_change_ma_connection_pays('', '', '')

### map chlor

In [45]:
w_pays_chl = v.Select(

    label = 'Pays pour chlo', 
    items = list_pays, 
    thumb_label='always', 
    v_model = "Netherlands",
    class_="px-6"
)

outputMapChl = Output()

In [46]:
world = chemin + "custom.geo.json"

def ChoroplethNbrLiaisons(pays,m):
    
    df_tmp = pd.merge(pd.DataFrame.from_dict(dict_aeroports,orient='index')['Country'],
            df_vols[df_vols["Depart"].isin(filtreDictAeroports('Country',pays))],
            left_index=True,
            right_on="Arrivee").groupby('Country').size()

    #errores = 'ignore' car certains pays n'ont pas de vol interne. (Cf Singapour)
    df_tmp.drop(pays, inplace=True, errors='ignore')

    f.Choropleth(
        geo_data=world,
        name="choropleth",
        data=df_tmp,
        key_on="feature.properties.name",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=.1,
        legend_name="Nombre de liaisons",
        nan_fill_opacity = 0.1,
        nan_fill_color='white',
        Highlight= True
    ).add_to(m)

def on_change_map_chl(widget, event, data):

    m = f.Map(
        location=[0, 0], 
        zoom_start=2,
        min_zoom  = 2,
        min_lot=-200,
        max_lot=200,
        min_lat=-90,
        max_lat=90,
        max_bounds=True,)

    ChoroplethNbrLiaisons(w_pays_chl.v_model,m)
    f.LayerControl().add_to(m)
    
    chl = Figure()
    chl.add_child(m)
    outputMapChl.clear_output(wait = True)

    with outputMapChl:
        display(chl)

w_pays_chl.on_event('change', on_change_map_chl)
on_change_map_chl('', '', '')

### box plot

In [47]:
w_listCompagnies = v.Select(
    items = list_Compagnies,
    v_model = 'Air France',
    multiple = True, 
    label = 'Choisi les compagnies', 
    thumb_label='always', 
    class_="px-6"
)


In [48]:
def calculDistLigne(ligne):
    dist = 0
    try : 
        dist = dict_route[frozenset((ligne.Depart,ligne.Arrivee))]['Dist']
    except IndexError:
        #Certains aéroports de la df_vols ne sont pas dans le dict...
        pass

    return dist

In [49]:
#df_volsDist = df_vols.copy()
#df_volsDist['Dist'] = df_volsDist.apply(calculDistLigne, axis=1)

In [50]:
'''
#On ajoute 10km pour corriger un bug d'affichage...
df_volsDist['Dist'] = df_volsDist['Dist'] + 10
fig = px.histogram(df_volsDist, x="Dist",nbins = 60, log_y=True, marginal="box" ,width=800, height=550, range_x=(0,16000),
                    title ="Histogramme des vols en fonction de la distance",
                    labels={"Dist" : "Distance (km)","count" : "Nbr de vols"})
fig.show()
'''

'\n#On ajoute 10km pour corriger un bug d\'affichage...\ndf_volsDist[\'Dist\'] = df_volsDist[\'Dist\'] + 10\nfig = px.histogram(df_volsDist, x="Dist",nbins = 60, log_y=True, marginal="box" ,width=800, height=550, range_x=(0,16000),\n                    title ="Histogramme des vols en fonction de la distance",\n                    labels={"Dist" : "Distance (km)","count" : "Nbr de vols"})\nfig.show()\n'

In [51]:
def ajoutColTypeDest(ligne, boolCompagnie : bool = False):
    """
        Fonction : Exprime le type de vol en fonction du pays d'origine OU du pays de la compagnie
                    Pays par defaut
        Retour : National, Continental ou International
    """
    if(boolCompagnie):
        dict_tmp = dict_compagnies
        col = ligne.Code_ICAO
    else:
        dict_tmp = dict_aeroports
        col = ligne.Depart

    try :
        if(dict_tmp[col]['Country'] == dict_aeroports[ligne.Arrivee]['Country']):
            TypeDest = 'National'
        elif(dict_pays[dict_tmp[col]['Country']] == dict_pays[dict_aeroports[ligne.Arrivee]['Country']] ):
            TypeDest = 'Continental'
        else:
            TypeDest = 'International'
        return TypeDest
    except:
        pass

In [52]:
#Boxplot plotly
'''
def boxplotDistanceParCompagnie(listCompagnie : list):
    #Filtre sur les compagnies
    df = df_volsDist[df_volsDist['Code_ICAO'].isin(listCompagnie)].copy()

    #On ajoute le type de destination
    df['TypeDest'] = df.apply(ajoutColTypeDest, axis=1)

    #On affiche
    fig = px.box(df,x='TypeDest',y='Dist', color=[dict_compagnies[x]['Name'] for x in df['Code_ICAO']],
                labels={"color" : "Compagnies", "TypeDest" : "Type de destination", "Dist" : "Distance (km)"},
                title="Boxplot des distances en fonction du type de destination",
                width=900,
                height=500)
    fig.show()

boxplotDistanceParCompagnie(['DLH','AFR','ACA']) '''


'\ndef boxplotDistanceParCompagnie(listCompagnie : list):\n    #Filtre sur les compagnies\n    df = df_volsDist[df_volsDist[\'Code_ICAO\'].isin(listCompagnie)].copy()\n\n    #On ajoute le type de destination\n    df[\'TypeDest\'] = df.apply(ajoutColTypeDest, axis=1)\n\n    #On affiche\n    fig = px.box(df,x=\'TypeDest\',y=\'Dist\', color=[dict_compagnies[x][\'Name\'] for x in df[\'Code_ICAO\']],\n                labels={"color" : "Compagnies", "TypeDest" : "Type de destination", "Dist" : "Distance (km)"},\n                title="Boxplot des distances en fonction du type de destination",\n                width=900,\n                height=500)\n    fig.show()\n\nboxplotDistanceParCompagnie([\'DLH\',\'AFR\',\'ACA\']) '

### description + titles pour les maps 

# Main affichage 

In [118]:
menu_choix = v.Layout(pa_4=True, _metadata={'mount_id': 'content-nav'}, column=True, expand_on_hover = True, children=[
        
        
        v.Html(tag = 'h4', children = ['Cartes : ']),
        v.ListItem(tabindex = 0 , href = '#carteLiasonAirport', children = ['Carte de connexion : liaisons depuis un aéroport']),
        v.ListItem(tabindex = 1 , href = '#carteLiasonnationales', children = [ 'Carte de connexion : liaisons nationales']),
        v.ListItem(tabindex = 2 , href = '#carteChoropletes', children = [ 'Cartes Choroplètes']),
        v.ListItem(tabindex = 3 , href = '#Boxplot', children = [ 'Boxplot']),
        
        
        v.Html(tag = 'h4', children = ['Graphes : ']),
        v.Html(tag = 'h4', children = ['Autres : ']),
        v.ListItem(tabindex = 4 , href = '#carteChoropletes', children = ['Correspondances entre deux airports'] ),
        

        ]) 

In [119]:
main =  v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
            v.Tab(),
            v.TabItem(children=[
                    v.Flex(row = True, children = [
                    v.Layout(
                        row=True, wrap=True, align_center=True, id = 'carteLiasonAirport', 
                        children=[
                                v.Flex(column = True, xs12=True, lg6=True, xl4 = True, children=[ 
                                    v.Html(tag = 'h2', class_ = 'pl-6', children = ['Carte de connexion : liaisons depuis un aéroport'] ),
                                    v.Html(tag = 'p' , class_ = 'pl-6', children = ['Cette carte permet de mettre en évidence les liaisons possibles entre un aéroport et le reste du monde.',
                                                                                    'Grâce au filtre par compagnie, il est possible d\'affiner la visualisation et ainsi voir tous les trajets ',
                                                                                    'd\'une compagnie depuis un aéroport.'] ),
                                    depart,compagnies,tousLesCompagnies  
                                ]),
                                v.Flex(column = True, xs12=True, lg6=True, xl4 = True, children=[
                                    outputMapConnection
                                ])
                        ]),
                    ]),
                    v.Flex(row = True, children = [v.Html(tag = 'br'), v.Divider(), v.Html(tag = 'br')]),
                    v.Flex(row = True, children = [
                        v.Layout(
                            row=True, wrap=True, align_center=True, id = 'carteLiasonnationales', 
                            children=[
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                        outputMapConnectionPays 
                                    ]),
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True, children=[
                                        v.Html(tag = 'h2', class_ = 'p-6', children = ['Carte de connexion : liaisons nationales'] ),
                                        v.Html(tag = 'p' , class_ = 'p-6', children = [' Cette carte permet de visualiser les hubs nationaux du pays sélectionné. La taille des',
                                                                                        ' marqueurs indique représente le flux aéroportuaire (Nombre de vols entrant + sortant) tandis',
                                                                                        ' que la largeur des lignes représente le flux sur la liaison. '] ),
                                        w_pays_li 
                                    ])
                            ]),
                    ]),
                    v.Flex(row = True, children = [v.Html(tag = 'br'), v.Divider(), v.Html(tag = 'br')]),
                    v.Flex(row = True, children = [
                        v.Layout(
                            row=True, wrap=True, align_center=True,  id = 'carteChoropletes', 
                            children=[
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                        v.Html(tag = 'h3', class_ = 'p-6', children = ['Cartes Choroplètes : liaisons internationales'] ),
                                        v.Html(tag = 'p' , class_ = 'p-6', children = ['Cette carte permet de représenter les liaisons entre le pays sélectionné et le reste du monde.']),
                                        v.Html(tag = 'p' , class_ = 'p-6', children = ['Remarque : Il est intéressant de voir que le passé colonial de chaque pays resurgit toujours au niveau de l\'aviation. ']),
                                        w_pays_chl
                                    ]),
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True, children=[
                                        outputMapChl
                                    ])
                            ]),
                    ]),
                    v.Flex(row = True, children = [v.Html(tag = 'br'), v.Divider(), v.Html(tag = 'br')]),
                    v.Flex(row = True, children = [
                        v.Layout(
                            row=True, wrap=True, align_center=True, 
                            children=[
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                        v.Html(tag = 'h3',id = 'Boxplot', class_ = 'p-6', children = ['Boxplot des distances des liaisons en fonction du type de destination'] ),
                                        
                                        w_listCompagnies
                                    ]),
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True, children=[
                                        
                                    ])
                            ]),
                    ]),
                    v.Flex(row = True, children = [v.Html(tag = 'br'), v.Divider(), v.Html(tag = 'br')]),                
                    v.Flex(row = True, children = [
                    v.Layout(
                        row=True, wrap=True, align_center=True,  
                        children=[
                                v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                    v.Html(tag = 'h3', class_ = 'px-6', children = ['TITLE2'] ),
                                    v.Html(tag = 'p', class_ = 'px-6', children = ['ici een description ici een description ici een description ici een description ici een description ici een description '] ),
                                    
                                ]),
                                v.Flex(column = True, xs12=True, lg6=True, xl4 = True, children=[
                                    airportD, airportA, nbAirports
                                ])
                        ]),
                    ]),
                    v.Flex(row = True, children = [
                        v.Layout(
                            row=True, wrap=True, align_center=True, 
                            children=[
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                        v.Html(tag = 'h4', class_ = 'text-center', align_center= True, children = ['Le chemin le plus court en terme de distance']),
                                        outputMAPCourt
                                    ]),
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                        v.Html(tag = 'h3', class_ = 'px-6', children = ['TITLE2'] ),
                                        v.Html(tag = 'p', class_ = 'px-6', children = ['ici een description ici een description ici een description ici een description ici een description ici een description '] ),
                                    ])
                            ]),
                    ]),
                    v.Flex(row = True, children = [
                        v.Layout(
                            row=True, wrap=True, align_center=True, 
                            children=[
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                        v.Html(tag = 'h4', class_ = 'text-center', align_center= True, children = ['Le ou les chemins avec {} airports entre l\'airport de debut et l\'airport d\'arrivee'.format(nbAirports.v_model - 2 )]),
                                        outputMAPPlusieurs
                                    ]),
                                    v.Flex(column = True, xs12=True, lg6=True, xl4 = True , children=[ 
                                        v.Html(tag = 'h3', class_ = 'px-6', children = ['TITLE2'] ),
                                        v.Html(tag = 'p', class_ = 'px-6', children = ['ici een description ici een description ici een description ici een description ici een description ici een description '] ),
                                    ])
                            ]),
                    ]),
            ]) 
        ])

# code from https://github.com/voila-dashboards/voila-vuetify/blob/master/bqplot_vuetify_example.ipynb

 voila --template vuetify-default  --enable_nbextensions=True --VoilaConfiguration.file_whitelist="['favicon.ico']" .\finaldashboard.ipynb

# Drawing graph 

In [55]:
# drawing_graph = dessinerGraphe(graph_totale) # 55s 

In [56]:
g = ig.Graph.TupleList(graph_totale.edges(), directed=True)
layout = g.layout("large")

# fait une fois --> maintenant en mémoire 
#ig.plot(g, "graph_t.png", shape = 'triangle-up', autocurve = True)

# Cas particulaire

In [57]:
n0 = list_airports_codes.index('LYS')
n1 = list_airports_codes.index('LBG')
n2 = list_airports_codes.index('FAB')
n3 = list_airports_codes.index('LAX')


w1 = graph_totale.get_edge_data(n3,n2)
w2 = graph_totale.get_edge_data(n1,n3)
#print(w1,w2)
print(graph_totale.get_edge_data(n0,n1))
print(graph_totale.get_edge_data(n1,n2))
print(graph_totale.get_edge_data(n2,n3))
print(graph_totale.get_edge_data(n0,n1)['weight'] + graph_totale.get_edge_data(n1,n2)['weight'] + graph_totale.get_edge_data(n2,n3)['weight'])

print("----------------------")
n4 = list_airports_codes.index('CDG')
n5 = list_airports_codes.index('LAX')
print(graph_totale.get_edge_data(n0,n4))
print(graph_totale.get_edge_data(n4,n5))
print(graph_totale.get_edge_data(n0,n4)['weight'] + graph_totale.get_edge_data(n4,n5)['weight'])


{'weight': 411.7061792235852}
{'weight': 344.33294832625165}
{'weight': 8752.142514126186}
9508.181641676023
----------------------
{'weight': 412.56144600362046}
{'weight': 9097.059799998087}
9509.621246001709
